In [1]:
    from unittest import TestCase, expectedFailure, skipUnless
    IPY = False
    try:
        from IPython import get_ipython
        from IPython.utils.capture import capture_output
        IPY = get_ipython and get_ipython()
    except: 
        get_ipython = None
    from pathlib import Path
    
    try:
        from importnb import *
    except:
        # install importnb if it doesn't exist, this makes the tests work on binder.
        __import__('subprocess').check_call("pip install importnb".split())
        from importnb import *

    __file__ = globals().get('__file__', 'unittests.ipynb')

In [10]:
    class TestExtension(TestCase): 
        def setUp(Test): load_ipython_extension()
                
        def test_import(Test):
            try:
                from . import import_this
            except:
                import import_this

            assert import_this.__file__.endswith('.ipynb')
            assert import_this.test is 42
            assert isinstance(import_this, __import__('types').ModuleType)
            
        @expectedFailure
        def test_failure(Test):
            try:
                from . import failure
            except:
                import failure
            
        def tearDown(Test): unload_ipython_extension()


In [3]:
    class TestContext(TestCase): 
        def setUp(Test): 
            with Notebook(): 
                try:
                    from . import import_this
                except:
                    import import_this
            Test.loader = import_this
                  
        def test_import(Test):
            loader = Test.loader
            assert loader.__file__.endswith('.ipynb')
            assert loader.test is 42
            assert isinstance(loader, __import__('types').ModuleType)
            
        @skipUnless(not IPY, """importnb is probably installed""")
        @expectedFailure
        def test_reload_without_context(Test):
            reload(Test.loader)
            
        def test_reload_with_context(Test):
            with Notebook(): assert reload(Test.loader)

In [4]:
    class TestPartial(TestCase): 
        def setUp(Test): 
            with Partial(capture=True): 
                try:
                    from . import failure
                except:
                    import failure
            Test.failure = failure
                  
        def test_exception(Test):
            assert Test.failure.a is 42
            assert isinstance(Test.failure.__exception__, BaseException)
            assert not hasattr(Test.failure, 'b')
            
        def test_traceback(Test):
            from traceback import print_tb
            from io import StringIO
            s = StringIO()
            print_tb(Test.failure.__exception__.__traceback__, file=s)
            assert """tests/failure.ipynb", line 22, in <module>\n    "    assert False\\n""" in s.getvalue(), """Traceback is not satisfied"""

In [5]:
    class TestRemote(TestCase): 
        @skipUnless(False and IPY
             and (
                Path(__file__).parent / 'PythonDataScienceHandbook').exists(), 
            """requires IP""")
        def test_imports(Test):
            with capture_output(), Notebook():
                try:
                    from .PythonDataScienceHandbook.notebooks import _02_The_Basics_Of_NumPy_Arrays as tutorial
                except:
                    from PythonDataScienceHandbook.notebooks import _02_The_Basics_Of_NumPy_Arrays as tutorial
            assert all(hasattr(tutorial, str) for str in ('grid', 'x', 'x1', 'x2'))

In [6]:
    from pytest import mark

In [7]:
    @mark.skipif(not IPY, reason="""We need ipython""")
    def test_readme():
        get_ipython().system('jupyter nbconvert --to markdown --execute --ExecutePreprocessor.kernel_name=python ../readme.ipynb')
        get_ipython().system('cd .. && ipython -m readme')

In [8]:
    if __name__ == '__main__':
        get_ipython().system('source activate p6 && python -m pytest ')

============================= test session starts ==============================
platform darwin -- Python 3.6.3, pytest-3.5.0, py-1.5.3, pluggy-0.6.0
benchmark: 3.1.1 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /Users/tonyfast/importnb, inifile:
plugins: cov-2.5.1, benchmark-3.1.1, hypothesis-3.56.5, importnb-0.2.3
collected 9 items                                                              

test_.ipynb x...x..ss                                                    [100%]

================ 5 passed, 2 skipped, 2 xfailed in 2.14 seconds ================
